In [14]:
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
from langchain.document_loaders import JSONLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY_4")
#print(os.getenv("GOOGLE_API_KEY_4"))

In [3]:
#print(openai_api_key)

folder = r"D:\my_code_profile\group_project\Verdant\Verdant\Rag_Base\jsonfiles\energy"
documents = []

print(f"Attempting to access folder: '{folder}'") # Add this line
if not os.path.exists(folder): # Add this check
    print(f"Error: Folder '{folder}' does not exist.")
else:
    print(f"Folder '{folder}' exists. Listing contents...")

for file in os.listdir(folder):
    if file.endswith(".json"):
        f = os.path.join(folder, file)
        loader= JSONLoader(file_path=f, jq_schema = ".", text_content=False )  
        documents.extend(loader.load())

print(f"Loaded {len(documents)} pages")

Attempting to access folder: 'D:\my_code_profile\group_project\Verdant\Verdant\Rag_Base\jsonfiles\energy'
Folder 'D:\my_code_profile\group_project\Verdant\Verdant\Rag_Base\jsonfiles\energy' exists. Listing contents...
Loaded 1 pages


In [10]:
faiss_index_dir = "faiss_index_energy"  # You can change this path as needed

embedding_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
if not os.path.exists(faiss_index_dir):
    os.makedirs(faiss_index_dir)
    print(f"Created FAISS index directory: {faiss_index_dir}")

faiss_main_file = os.path.join(faiss_index_dir, "index.faiss")
faiss_pkl_file = os.path.join(faiss_index_dir, "index.pkl")

if os.path.exists(faiss_main_file) and os.path.exists(faiss_pkl_file):
    # If index exists, load it
    vectorstore = FAISS.load_local(faiss_index_dir, embedding_model, allow_dangerous_deserialization=True)
    print("FAISS Document Index Loaded from existing directory.")
else:
    # If index does not exist, create it and save it
    print("Creating and saving FAISS Document Index...")
    vectorstore = FAISS.from_documents(documents, embedding_model)
    vectorstore.save_local(faiss_index_dir)
    print("FAISS Document Index Created and Saved.")

retriever = vectorstore.as_retriever()


Created FAISS index directory: faiss_index_energy
Creating and saving FAISS Document Index...
FAISS Document Index Created and Saved.


In [11]:

template = '''
Chat History : {chat_history}
Context : {context}
Question : {question}
Instructions:
1. For the Energy tracker you have to use the below context:
-> Apply the appropriate formula to calculate the CO₂ amount based on the question
use this formula: Total = quantity * emission factor,  where user will give quantity and material type. from that get average weight from context.
In the json file all the energy with there unit and there emission factor is given use that .
In the quantity the user will give input as number and then multiply it to emission factor of that energy
-> Output the CO₂ amount (in kg).
-> Also give 2 suggestions to reduce this emission.

'''
custom_prompt = PromptTemplate.from_template(template)

# --- Memory ---
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [12]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7, google_api_key=google_api_key)

conversational_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever = retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": custom_prompt},
    return_source_documents=False 
)

In [13]:
#print(os.getenv("GOOGLE_API_KEY"))
print("\n--- RAG Chatbot (with Conversation History) ---")
print("Type 'exit', 'quit', or 'bye' to end the conversation.")

while True:
    try:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit", "bye"]:
            print("Bot: Goodbye!")
            break

 
        response = conversational_qa_chain({"question": user_query})
        print(f"Bot: {response['answer']}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break


--- RAG Chatbot (with Conversation History) ---
Type 'exit', 'quit', or 'bye' to end the conversation.
Bot: You can calculate the **total CO2 emissions (in kg)** for various energy sources.

The inputs required are:
1.  The **quantity** (a number) of the energy source consumed.
2.  The **type of energy source** (e.g., electricity, LPG, PNG, petrol, diesel, CNG, firewood, charcoal, or cow dung).
Bot: For 10 liters of petrol, the CO2 emissions would be:

*   **Petrol Emission Factor:** 2.31 kg CO2e per liter
*   **Quantity:** 10 liters

**Calculation:** 10 liters * 2.31 kg CO2e/liter = **23.1 kg CO2e**

**Suggestions to reduce this emission:**
1.  **Opt for public transportation, carpooling, cycling, or walking** for your commute and errands whenever possible.
2.  **Maintain your vehicle regularly** to ensure optimal fuel efficiency, and consider switching to more fuel-efficient or electric vehicles in the future.
Bot: For 10 liters of petrol, the CO2 emissions would be:

*   **Petrol E